# Universal MLX Model Converter

This notebook converts any Hugging Face model to MLX format and uploads it to Hugging Face.

## Features
- Convert any compatible model to MLX format
- Automatic model download and organization
- Configurable quantization options
- Automatic upload to Hugging Face

## Requirements
- macOS with Apple Silicon (M1/M2/M3/M4)
- Python 3.9+
- Sufficient disk space (varies by model size)
- Hugging Face account and token

## Step 1: Configuration - Enter Your Model Details

In [ ]:
# Model Configuration
print("=== Universal MLX Model Converter ===")
print("Please enter the details for the model you want to convert:\n")

# Get user inputs
SOURCE_MODEL = input("Enter the source model name (e.g., 'microsoft/DialoGPT-medium'): ").strip()
TARGET_REPO = input("Enter your target repository name (e.g., 'username/model-name-mlx'): ").strip()
HF_USERNAME = input("Enter your Hugging Face username: ").strip()

# Quantization options
print("\n=== Quantization Options ===")
print("1. No quantization (largest size, best quality)")
print("2. 4-bit quantization (recommended, good balance)")
print("3. 8-bit quantization (larger than 4-bit, better quality)")
quant_choice = input("Choose quantization option (1-3): ").strip()

# Set quantization parameters
if quant_choice == "2":
    USE_QUANTIZATION = True
    Q_BITS = 4
    Q_GROUP_SIZE = 64
elif quant_choice == "3":
    USE_QUANTIZATION = True
    Q_BITS = 8
    Q_GROUP_SIZE = 128
else:
    USE_QUANTIZATION = False
    Q_BITS = None
    Q_GROUP_SIZE = None

print(f"\n=== Configuration Summary ===")
print(f"Source Model: {SOURCE_MODEL}")
print(f"Target Repository: {TARGET_REPO}")
print(f"Username: {HF_USERNAME}")
print(f"Quantization: {'Yes' if USE_QUANTIZATION else 'No'}")
if USE_QUANTIZATION:
    print(f"  - Bits: {Q_BITS}")
    print(f"  - Group Size: {Q_GROUP_SIZE}")

confirm = input("\nProceed with these settings? (y/n): ").strip().lower()
if confirm != 'y':
    print("Configuration cancelled. Please restart and enter new values.")
    import sys
    sys.exit(1)
else:
    print("✅ Configuration confirmed!")

## Step 2: Environment Setup

In [ ]:
import os
import sys
import subprocess
from pathlib import Path
import re

# Create safe directory names from model names
def create_safe_dirname(name):
    """Convert model name to safe directory name"""
    return re.sub(r'[^a-zA-Z0-9_-]', '_', name.replace('/', '_'))

# Set up project directories
project_dir = Path.cwd()
models_dir = project_dir / "models"
source_model_dirname = create_safe_dirname(SOURCE_MODEL)
target_model_dirname = create_safe_dirname(TARGET_REPO.split('/')[-1])  # Use just the model name part

original_model_dir = models_dir / source_model_dirname
mlx_model_dir = models_dir / f"{target_model_dirname}_mlx"

# Create directories
models_dir.mkdir(exist_ok=True)
original_model_dir.mkdir(exist_ok=True)
mlx_model_dir.mkdir(exist_ok=True)

print(f"Project directory: {project_dir}")
print(f"Models directory: {models_dir}")
print(f"Source model directory: {original_model_dir}")
print(f"MLX model directory: {mlx_model_dir}")

# Environment setup
print("\nSetting up environment...")
try:
    subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip"], check=True, capture_output=True)
    subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "numpy"], check=True, capture_output=True)
    print("✅ Core packages updated successfully")
except Exception as e:
    print(f"⚠️ Package update warning: {e}")

## Step 3: Install Dependencies

In [ ]:
# Install required packages
print("Installing MLX and dependencies...")

packages = [
    "mlx-lm",
    "transformers",
    "torch", 
    "huggingface_hub",
    "datasets",
    "accelerate",
    "sentencepiece",
    "protobuf"
]

failed_packages = []
for package in packages:
    try:
        print(f"Installing {package}...")
        result = subprocess.run([sys.executable, "-m", "pip", "install", package], 
                              check=True, capture_output=True, text=True)
        print(f"✅ {package} installed successfully")
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to install {package}: {e}")
        failed_packages.append(package)

if failed_packages:
    print(f"\n⚠️ Failed to install: {', '.join(failed_packages)}")
    print("You may need to install these manually or restart the kernel.")
else:
    print("\n🎉 All packages installed successfully!")

## Step 4: Test Imports

In [ ]:
# Test imports
print("Testing imports...")
import_success = True

try:
    import numpy as np
    print("✅ numpy imported successfully")
except ImportError as e:
    print(f"❌ numpy import failed: {e}")
    import_success = False

if import_success:
    try:
        import mlx.core as mx
        print("✅ mlx.core imported successfully")
    except ImportError as e:
        print(f"❌ mlx.core import failed: {e}")
        import_success = False

    try:
        from mlx_lm import convert, load, generate
        print("✅ mlx_lm imported successfully")
    except ImportError as e:
        print(f"❌ mlx_lm import failed: {e}")
        import_success = False

    try:
        from huggingface_hub import login, HfApi, snapshot_download, upload_folder
        print("✅ huggingface_hub imported successfully")
    except ImportError as e:
        print(f"❌ huggingface_hub import failed: {e}")
        import_success = False

if import_success:
    print("\n🎉 All imports successful! Ready to proceed.")
else:
    print("\n⚠️ Some imports failed. Please restart kernel and try again.")

## Step 5: Hugging Face Authentication

In [ ]:
import getpass

# Login to Hugging Face
print("Please enter your Hugging Face token:")
print("(You can get it from: https://huggingface.co/settings/tokens)")
hf_token = getpass.getpass("HF Token: ")

try:
    login(token=hf_token)
    print("✅ Successfully logged in to Hugging Face!")
    
    # Initialize HF API
    api = HfApi()
    
    # Test API access
    user_info = api.whoami()
    print(f"Logged in as: {user_info['name']}")
    
except Exception as e:
    print(f"❌ Failed to login: {e}")
    print("Please check your token and try again.")

## Step 6: Download Source Model

In [ ]:
import shutil
from datetime import datetime

print(f"Downloading {SOURCE_MODEL}...")
print(f"This may take a while depending on model size and internet connection.")
print(f"Download location: {original_model_dir}")

# Check if model already exists
if list(original_model_dir.glob("*")):
    print(f"Model files found in {original_model_dir}")
    redownload = input("Re-download the model? (y/n): ").strip().lower()
    if redownload == 'y':
        shutil.rmtree(original_model_dir)
        original_model_dir.mkdir(exist_ok=True)
    else:
        print("Using existing model files.")

if not list(original_model_dir.glob("*")):
    try:
        start_time = datetime.now()
        
        # Download the model
        downloaded_path = snapshot_download(
            repo_id=SOURCE_MODEL,
            local_dir=str(original_model_dir),
            local_dir_use_symlinks=False,
            resume_download=True
        )
        
        end_time = datetime.now()
        duration = end_time - start_time
        
        print(f"✅ Model downloaded successfully in {duration}")
        print(f"Download location: {downloaded_path}")
        
    except Exception as e:
        print(f"❌ Download failed: {e}")
        print("Please check the model name and your internet connection.")

# List downloaded files
print("\nDownloaded files:")
total_size = 0
for file in original_model_dir.glob("*"):
    if file.is_file():
        size_mb = file.stat().st_size / 1024 / 1024
        total_size += size_mb
        print(f"  {file.name} ({size_mb:.2f} MB)")

print(f"\nTotal model size: {total_size:.2f} MB")

## Step 7: Convert to MLX Format

In [ ]:
import shutil
from datetime import datetime

print("Starting MLX conversion...")
print(f"Source: {original_model_dir}")
print(f"Destination: {mlx_model_dir}")
print(f"Quantization: {'Enabled' if USE_QUANTIZATION else 'Disabled'}")

# Clean up existing MLX directory
if mlx_model_dir.exists() and list(mlx_model_dir.glob("*")):
    print(f"Cleaning existing MLX directory: {mlx_model_dir}")
    shutil.rmtree(mlx_model_dir)
    mlx_model_dir.mkdir(exist_ok=True)

conversion_success = False
start_time = datetime.now()

# Try different conversion methods
methods = [
    {"name": "Standard with quantization", "func": lambda: convert(
        str(original_model_dir), 
        str(mlx_model_dir),
        quantize=USE_QUANTIZATION,
        q_group_size=Q_GROUP_SIZE,
        q_bits=Q_BITS
    ) if USE_QUANTIZATION else None},
    {"name": "Standard without quantization", "func": lambda: convert(
        str(original_model_dir), 
        str(mlx_model_dir)
    )},
    {"name": "Command line conversion", "func": lambda: subprocess.run([
        sys.executable, "-m", "mlx_lm.convert",
        "--hf-path", str(original_model_dir),
        "--mlx-path", str(mlx_model_dir)
    ] + (["--quantize"] if USE_QUANTIZATION else []), check=True)}
]

for method in methods:
    if method["func"] is None:  # Skip if quantization method not applicable
        continue
        
    try:
        print(f"\nTrying: {method['name']}...")
        method["func"]()
        print(f"✅ {method['name']} completed successfully!")
        conversion_success = True
        break
    except Exception as e:
        print(f"❌ {method['name']} failed: {e}")
        continue

end_time = datetime.now()
duration = end_time - start_time

if conversion_success:
    print(f"\n🎉 MLX conversion completed successfully in {duration}!")
    
    # Verify conversion results
    print("\nConverted MLX files:")
    total_size = 0
    for file in mlx_model_dir.glob("*"):
        if file.is_file():
            size_mb = file.stat().st_size / 1024 / 1024
            total_size += size_mb
            print(f"  {file.name} ({size_mb:.2f} MB)")
    
    print(f"\nTotal MLX model size: {total_size:.2f} MB")
else:
    print(f"\n❌ All conversion methods failed. Please check the model compatibility.")

## Step 8: Test MLX Model

In [ ]:
if conversion_success:
    print("Testing the converted MLX model...")
    
    try:
        # Load the converted MLX model
        model, tokenizer = load(str(mlx_model_dir))
        print("✅ MLX model loaded successfully!")
        
        # Test generation with a simple prompt
        test_prompt = "Hello, this is a test"
        print(f"\nTesting with prompt: '{test_prompt}'")
        
        response = generate(
            model, 
            tokenizer, 
            prompt=test_prompt, 
            max_tokens=50,
            temp=0.7
        )
        
        print(f"Generated response: {response}")
        print("\n✅ MLX model is working correctly!")
        
        model_test_success = True
        
    except Exception as e:
        print(f"❌ Error testing MLX model: {e}")
        print("The model converted but may have compatibility issues.")
        model_test_success = False
else:
    print("Skipping model test due to conversion failure.")
    model_test_success = False

## Step 9: Create Model Documentation

In [ ]:
if conversion_success:
    from datetime import datetime
    
    # Create model card
    model_card_content = f"""---
license: apache-2.0
base_model: {SOURCE_MODEL}
tags:
- mlx
- converted
- apple-silicon
{'- quantized' if USE_QUANTIZATION else ''}
---

# {TARGET_REPO.split('/')[-1]}

This is an MLX-optimized version of [{SOURCE_MODEL}](https://huggingface.co/{SOURCE_MODEL}), converted for Apple Silicon devices.

## Model Details

- **Base Model**: {SOURCE_MODEL}
- **Conversion Date**: {datetime.now().strftime('%Y-%m-%d')}
- **Format**: MLX
- **Optimization**: Optimized for Apple Silicon (M1/M2/M3/M4)
- **Quantization**: {'Yes (' + str(Q_BITS) + '-bit)' if USE_QUANTIZATION else 'No'}

## Usage

```python
from mlx_lm import load, generate

# Load the model
model, tokenizer = load("{TARGET_REPO}")

# Generate text
response = generate(
    model, 
    tokenizer, 
    prompt="Your prompt here", 
    max_tokens=100,
    temp=0.7
)
print(response)
```

## Requirements

- macOS with Apple Silicon
- MLX framework: `pip install mlx-lm`

## Performance

This MLX version is optimized for Apple Silicon and should provide:
- Faster inference on Mac devices
- Lower memory usage
- Better integration with macOS

## License

This model follows the same license as the original model: {SOURCE_MODEL}

## Conversion Details

- Converted using mlx-lm
- {'Quantized to ' + str(Q_BITS) + '-bit precision' if USE_QUANTIZATION else 'No quantization applied'}
- Tested and verified on Apple Silicon
"""
    
    # Save model card
    readme_path = mlx_model_dir / "README.md"
    with open(readme_path, "w", encoding="utf-8") as f:
        f.write(model_card_content)
    
    print("✅ Model documentation created successfully!")
    print(f"README.md saved to: {readme_path}")
else:
    print("Skipping documentation creation due to conversion failure.")

## Step 10: Upload to Hugging Face

In [ ]:
if conversion_success:
    print(f"Preparing to upload to: {TARGET_REPO}")
    
    # Create repository
    try:
        repo_url = api.create_repo(
            repo_id=TARGET_REPO,
            repo_type="model",
            exist_ok=True,
            private=False  # Set to True if you want a private repo
        )
        print(f"✅ Repository {TARGET_REPO} created/confirmed!")
        print(f"Repository URL: {repo_url}")
        
    except Exception as e:
        print(f"❌ Repository creation failed: {e}")
        print("Please check your repository name and permissions.")
        upload_success = False
    else:
        # Upload the model
        print(f"\nUploading MLX model to {TARGET_REPO}...")
        print("This may take a while depending on model size and internet connection.")
        
        try:
            start_time = datetime.now()
            
            upload_folder(
                folder_path=str(mlx_model_dir),
                repo_id=TARGET_REPO,
                repo_type="model",
                commit_message=f"Add MLX-converted {SOURCE_MODEL.split('/')[-1]} model",
                ignore_patterns=[".DS_Store", "*.pyc", "__pycache__", ".git"]
            )
            
            end_time = datetime.now()
            duration = end_time - start_time
            
            print(f"\n✅ Model successfully uploaded in {duration}!")
            print(f"🔗 Model URL: https://huggingface.co/{TARGET_REPO}")
            upload_success = True
            
        except Exception as e:
            print(f"❌ Upload failed: {e}")
            print("You may need to check your internet connection or Hugging Face permissions.")
            upload_success = False
else:
    print("Skipping upload due to conversion failure.")
    upload_success = False

## Step 11: Final Verification

In [ ]:
if upload_success:
    # Verify the upload
    try:
        repo_info = api.repo_info(repo_id=TARGET_REPO, repo_type="model")
        print(f"✅ Repository verified: {repo_info.id}")
        print(f"🔗 Repository URL: https://huggingface.co/{TARGET_REPO}")
        print(f"📅 Last modified: {repo_info.last_modified}")
        
        # List files in the repository
        files = api.list_repo_files(repo_id=TARGET_REPO, repo_type="model")
        print(f"\n📁 Files in repository ({len(files)} total):")
        for file in sorted(files):
            print(f"  📄 {file}")
            
        verification_success = True
        
    except Exception as e:
        print(f"❌ Verification failed: {e}")
        verification_success = False
else:
    print("Skipping verification due to upload failure.")
    verification_success = False

## Step 12: Cleanup Options

In [ ]:
# Optional cleanup
print("=== Cleanup Options ===")
print(f"Original model location: {original_model_dir}")
print(f"MLX model location: {mlx_model_dir}")

if verification_success:
    cleanup_choice = input("\nWhat would you like to clean up?\n1. Keep all files\n2. Delete original model only\n3. Delete both original and MLX models\nChoice (1-3): ").strip()
    
    if cleanup_choice == "2":
        try:
            shutil.rmtree(original_model_dir)
            print(f"✅ Original model deleted: {original_model_dir}")
        except Exception as e:
            print(f"❌ Failed to delete original model: {e}")
    
    elif cleanup_choice == "3":
        try:
            shutil.rmtree(original_model_dir)
            shutil.rmtree(mlx_model_dir)
            print(f"✅ Both models deleted from local storage")
        except Exception as e:
            print(f"❌ Failed to delete models: {e}")
    
    else:
        print("✅ All files kept locally")
else:
    print("Cleanup skipped due to incomplete conversion/upload process.")

## Step 13: Summary Report

In [ ]:
# Final summary
print("\n" + "="*60)
print("🎉 UNIVERSAL MLX CONVERTER - FINAL REPORT")
print("="*60)

print(f"\n📋 Configuration:")
print(f"   Source Model: {SOURCE_MODEL}")
print(f"   Target Repository: {TARGET_REPO}")
print(f"   Quantization: {'Yes (' + str(Q_BITS) + '-bit)' if USE_QUANTIZATION else 'No'}")

print(f"\n✅ Process Status:")
print(f"   Model Download: {'✅ Success' if 'downloaded_path' in locals() else '❌ Failed'}")
print(f"   MLX Conversion: {'✅ Success' if conversion_success else '❌ Failed'}")
print(f"   Model Testing: {'✅ Success' if model_test_success else '❌ Failed/Skipped'}")
print(f"   HF Upload: {'✅ Success' if upload_success else '❌ Failed/Skipped'}")
print(f"   Verification: {'✅ Success' if verification_success else '❌ Failed/Skipped'}")

if verification_success:
    print(f"\n🔗 Your converted model is available at:")
    print(f"   https://huggingface.co/{TARGET_REPO}")
    
    print(f"\n🚀 Usage Instructions:")
    print(f"   ```python")
    print(f"   from mlx_lm import load, generate")
    print(f"   model, tokenizer = load('{TARGET_REPO}')")
    print(f"   response = generate(model, tokenizer, prompt='Hello', max_tokens=100)")
    print(f"   ```")

print(f"\n" + "="*60)
print("Thank you for using the Universal MLX Converter!")
print("="*60)